1. Постройте нейронную сеть (берем несложную полносвязную сеть, меняем число слоев, число нейронов, типы активации, тип оптимизатора) на датасете from sklearn.datasets import load_boston.

2. Постройте 10-15 вариантов разных нейронных сетей и сведите результаты их работы в таблицу. Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Sequential, Model
from keras import backend

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
boston_dataset = load_boston()
boston_dataset.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [3]:
X = pd.DataFrame(boston_dataset.data, columns = boston_dataset.feature_names)
y = boston_dataset.target
X.head(2)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.9,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14


In [4]:
X = StandardScaler().fit_transform(X)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=1)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((379, 13), (379,), (127, 13), (127,))

In [6]:
# коэффициент детерминации для используемых моделей
def det_coeff(y_true, y_pred):
    SS_res =  backend.sum(backend.square(y_true-y_pred))
    SS_tot = backend.sum(backend.square(y_true - backend.mean(y_true)))
    return (1 - SS_res / (SS_tot + backend.epsilon()))

In [7]:
%%time
res = pd.DataFrame(columns=['n', 'layers', 'optimizer', 'activation', 'det_coeff', 'val_det_coeff'])

n_list = [5, 10, 20, 50]
layers_list = [3, 5, 7, 10]
optims = ['adam', 'RMSprop']
f_activ = ['relu', 'sigmoid', 'tanh']

# sp = 0
# fig = plt.figure(figsize=(120,20))
# plt.subplots_adjust(hspace=0.5)

for n in n_list:
    for layers in layers_list:
        for optim in optims:
            for f in f_activ:
#                 sp +=1
                
                model = Sequential([
                    Dense(n, input_shape=(13,), activation=f)
                ])
                for _ in range(layers - 1):
                    model.add(Dense(n, activation=f))
                model.add(Dense(1))
                
                model.compile(optimizer=optim, loss='mse', metrics=[det_coeff])
                
                history = model.fit(X_train, y_train,
                                    epochs=15,
                                    batch_size=10,
                                    verbose=0,
                                    validation_data=(X_test, y_test))
                
                train_loss, train_metric = model.evaluate(X_train, y_train)
                test_loss, test_metric = model.evaluate(X_test, y_test)
                
#                 ax = fig.add_subplot(48, 2, sp)
#                 ax.plot(history.history['val_det_coeff'], label=f'val {optim} {round(test_metric,2)}')
#                 ax.plot(history.history['det_coeff'], label=f'train {optim} {round(train_metric,2)}')
#                 ax.legend()
#                 ax.set_xlabel('epoch')
#                 ax.set_ylabel('det_coeff')
#                 ax.set(title = f'Ошибка сети от эпох при {n} нейронах и функции активации {f}')
                
                res = res.append({'n': n, 'layers': layers, 'optimizer': optim, 'activation': f,
                                  'det_coeff': train_metric, 'val_det_coeff': test_metric}, ignore_index=True)
# plt.show()

12/12 [==============================] - 0s 2ms/step - loss: 18.6037 - det_coeff: 0.7564


12/12 [==============================] - 0s 2ms/step - loss: 95.5504 - det_coeff: -0.2314


4/4 [==============================] - 0s 2ms/step - loss: 44.0484 - det_coeff: 0.5436
Wall time: 5min 22s


In [11]:
res.sort_values(by='val_det_coeff', ascending=False).head(20)

,n,layers,optimizer,activation,det_coeff,val_det_coeff
84,50,7,adam,relu,0.873804,0.900595
90,50,10,adam,relu,0.910458,0.898700
78,50,5,adam,relu,0.881320,0.895941
72,50,3,adam,relu,0.857101,0.893605
66,20,10,adam,relu,0.867114,0.888027
87,50,7,RMSprop,relu,0.873240,0.882909
75,50,3,RMSprop,relu,0.821293,0.877757
63,20,7,RMSprop,relu,0.854420,0.875008
69,20,10,RMSprop,relu,0.862395,0.873815
60,20,7,adam,relu,0.836463,0.865254


In [12]:
res[res['optimizer']=='adam'].sort_values(by='val_det_coeff', ascending=False).head(10)

,n,layers,optimizer,activation,det_coeff,val_det_coeff
84,50,7,adam,relu,0.873804,0.900595
90,50,10,adam,relu,0.910458,0.898700
78,50,5,adam,relu,0.881320,0.895941
72,50,3,adam,relu,0.857101,0.893605
66,20,10,adam,relu,0.867114,0.888027
60,20,7,adam,relu,0.836463,0.865254
54,20,5,adam,relu,0.834327,0.865014
48,20,3,adam,relu,0.776375,0.846431
42,10,10,adam,relu,0.785577,0.843407
30,10,5,adam,relu,0.754523,0.807641


In [13]:
res[res['optimizer']=='RMSprop'].sort_values(by='val_det_coeff', ascending=False).head(10)

,n,layers,optimizer,activation,det_coeff,val_det_coeff
87,50,7,RMSprop,relu,0.873240,0.882909
75,50,3,RMSprop,relu,0.821293,0.877757
63,20,7,RMSprop,relu,0.854420,0.875008
69,20,10,RMSprop,relu,0.862395,0.873815
81,50,5,RMSprop,relu,0.826730,0.855623
57,20,5,RMSprop,relu,0.813559,0.855015
39,10,7,RMSprop,relu,0.767354,0.849981
93,50,10,RMSprop,relu,0.819936,0.848670
51,20,3,RMSprop,relu,0.751740,0.845505
45,10,10,RMSprop,relu,0.756407,0.819570


Функция активации relu лучше всех. Большее число нейронов в слоях показывает лучший результат. По количеству слоев закономеронстей не видно. И 5, и 10 слоев достаточно для хорошей метрики. Если посмотреть на лучшие результаты для обоих оптимизаторов, то у adam метрика на валидационных данных выше, но у RMSprop получилось меньше переобучение. На тренировочных данных метрики слабо отличаются.